In [ ]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(ggplot2, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(sf, warn.conflicts = FALSE)
library(zeallot, warn.conflicts = FALSE)
library(stars, warn.conflicts = FALSE)

source("src/paths/paths.R")
source("src/load/load.R")
source("src/analysis/data/clim_availability.R")
source("notebooks/integrazioni_regionali/nb_tools/state_avail.R")
source("notebooks/integrazioni_regionali/nb_tools/pairing_procedure.R")

options(repr.plot.width = 9, repr.plot.res = 300)
path.piemonte <- file.path(path.ds, "ARPA", "PIEMONTE")
path.arpa.metadata <- file.path(path.piemonte, "metadata_deduped.geojson")
path.arpa.data <- file.path(path.piemonte, "dataset")
dem <- read_stars("temp/dem/dem30.tif")


In [ ]:
arpa.metadata <- read_sf(path.arpa.metadata) |> prepare_metadata(dem)
arpa.metadata <- bind_rows(T_MAX = arpa.metadata, T_MIN = arpa.metadata, .id = "variable")
arpa.ds <- open_dataset(path.arpa.data, format = "feather") |> select(-tclasse)
scia.metadata <- open.dataset("SCIA", "metadata") |> filter(state == "Piemonte")
scia.ds <- open.dataset("SCIA", "data") |> semi_join(scia.metadata, by = "identifier")
scia.metadata <- collect(scia.metadata) |>
    st_md_to_sf() |>
    prepare_metadata(dem)


In [ ]:
plot_state_avail(bind_rows(
    ARPA = arpa.ds |> collect(),
    SCIA = scia.ds |> mutate(identifier = cast(identifier, utf8())) |> collect(),
    .id = "db"
))


In [ ]:
scia.metadata <- scia.metadata |>
    group_by(variable, anagrafica) |>
    arrange(first_date, .by_group = TRUE) |>
    slice_tail() |>
    ungroup()
matches <- matches_table(scia.metadata, arpa.metadata, dist_km = 0.5)


In [ ]:
matches |>
    filter((anagrafica.x != str_remove_all(anagrafica.y, "'") & (anagrafica.x != comune))) |>
    select(starts_with("anag"), rete, comune, variable)


In [ ]:
c(data.scia.tmax, data.scia.tmin) %<-% (open.dataset("SCIA", "data") |>
    widen_split_data(matches, identifier.x))

c(data.arpa.tmax, data.arpa.tmin) %<-% (arpa.ds |>
    widen_split_data(matches, identifier.y))


In [ ]:
diffs <- compute_diffs(matches |> filter(variable == "T_MIN"), data.scia.tmin, data.arpa.tmin)


In [ ]:
thresh <- c(0.01, 0.05, 0.1, 0.5, 0.9, 0.95, 0.99)
diffs |> reframe(diffs = quantile(diffs, probs = thresh, na.rm = TRUE), quant = thresh)


In [ ]:
scia.metadata |>
    select(variable, rete, anagrafica, identifier, ends_with("date")) |>
    slice_head(n = 10)


In [ ]:
diffs |>
    left_join(scia.metadata |> filter(variable == "T_MAX") |> select(identifier, rete, anagrafica) |> mutate(identifier = as.character(identifier)), by = join_by(identifier.x == identifier)) |>
    filter(abs(diffs) > 3)


In [ ]:
analysis <- build_analysis(matches, data.scia.tmin, data.scia.tmax, data.arpa.tmin, data.arpa.tmax)


In [ ]:
analysis |> sensible_columns()


In [ ]:
arpa.data <- collect(arpa.ds) |> as_tsibble(index = date, key = c("identifier", "variable"))


In [ ]:
clim_availablility <- is_climatology_computable(arpa.data |> group_by_key(), value, as.Date("2000-01-01"), as.Date("2022-12-31"))
plot_clim_availability(clim_availablility, arpa.metadata) + theme(title = text("From 2000-01-01"))

In [ ]:
n_available_series(arpa.data, as.Date("2000-01-01"), as.Date("2022-12-31"))

In [ ]:
n_available_series(arpa.data, as.Date("2002-01-01"), as.Date("2022-12-31"))

In [ ]:
scia.internal_matches <- matches_table(scia.metadata, scia.metadata, dist_km = 0.5) |>
    filter(identifier.x > identifier.y) |>
    mutate(match_id = row_number())


In [ ]:
scia.analysis <- build_analysis(
    scia.internal_matches |>
        mutate(across(starts_with("identi"), ~ as.character(.))) |>
        select(starts_with("identif"), starts_with("anag"), starts_with("elev"), variable, starts_with("dem"), distance, match_id),
    data.scia.tmin, data.scia.tmax, data.scia.tmin, data.scia.tmax
)


In [ ]:
scia.analysis |> sensible_columns()


In [ ]:
scia.merged.tmin <- update_left(scia.analysis |> filter(variable == "T_MIN"), data.scia.tmin, data.scia.tmin) |> left_join(scia.internal_matches |> select(variable, starts_with("identif"), match_id) |> mutate(match_id = as.character(match_id)), by = "match_id", relationship = "many-to-one")
scia.merged.tmax <- update_left(scia.analysis |> filter(variable == "T_MAX"), data.scia.tmax, data.scia.tmax) |>
    left_join(scia.internal_matches |> select(variable, starts_with("identif"), match_id) |> mutate(match_id = as.character(match_id)), by = "match_id", relationship = "many-to-one")


In [ ]:
scia.merged.tmin |> filter(from.dpc)


In [ ]:
scia.merged.tmin |>
    filter(match_id < 50) |>
    ggplot(aes(date, value, color = from.dpc)) +
    geom_line()


In [ ]:
arpa.internal_matches <- matches_table(arpa.metadata, arpa.metadata, dist_km = 0.01) |>
    filter(data_inizio_sensore.x != data_inizio_sensore.y) |>
    mutate(match_id = row_number())


In [ ]:
arpa.internal_matches |> select(starts_with("identi"), starts_with("anag"))


In [ ]:
arpa.ds |> rename(identifier.x = identifier) |> mutate(db.x = "ARPA Piemonte", )

In [ ]:
open_dataset("db/merged", format = "feather")

In [ ]:
write_arpa_for_merge(arpa.ds |> collect(), "Piemonte")